<a href="https://colab.research.google.com/github/sc-AhmedAttia/ObjectDetection/blob/main/Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p output/{ssd,yolo} tools

In [ ]:
%%writefile tools/__init__.py
# init

Writing tools/__init__.py


In [ ]:
%env KAGGLE_USERNAME=datadinosaur
%env KAGGLE_KEY=xxxxxxxxxxxxxx

In [ ]:
!kaggle datasets download -d chetankv/dogs-cats-images

 98% 427M/435M [00:03<00:00, 117MB/s]
100% 435M/435M [00:03<00:00, 120MB/s]


In [ ]:
!unzip -qq dogs-cats-images.zip

In [ ]:
!rm -rf "/content/dog vs cat"

In [ ]:
!rm dogs-cats-images.zip

In [ ]:
%%writefile tools/config.py
# import the necessary packages
import torch
import os
# define the root directory followed by the test dataset paths
BASE_PATH = "dataset"
TEST_PATH = os.path.join(BASE_PATH, "test_set")
# specify image size and batch size
IMAGE_SIZE = 300
PRED_BATCH_SIZE = 4
# specify threshold confidence value for ssd detections
THRESHOLD = 0.50
# determine the device type 
DEVICE = torch.device("cuda") if torch.cuda.is_available() else "cpu"
# define paths to save output 
OUTPUT_PATH = "output"
SSD_OUTPUT = os.path.join(OUTPUT_PATH, "ssd")
YOLO_OUTPUT = os.path.join(OUTPUT_PATH, "yolo")

Writing tools/config.py


In [ ]:
%%writefile tools/data_utils.py
# import the necessary packages
from torch.utils.data import DataLoader
def get_dataloader(dataset, batchSize, shuffle=True):
	# create a dataloader and return it
	dataLoader= DataLoader(dataset, batch_size=batchSize,
		shuffle=shuffle)
	return dataLoader

def normalize(image, mean=128, std=128):
    # normalize the SSD input and return it 
    image = (image * 256 - mean) / std
    return image

Writing tools/data_utils.py


In [ ]:
# import necessary packages
from tools.data_utils import get_dataloader
import tools.config as config
from torchvision.transforms import Compose, ToTensor, Resize
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import cv2
import os
# initialize test transform pipeline
testTransform = Compose([
	Resize((config.IMAGE_SIZE, config.IMAGE_SIZE)), ToTensor()])
# create the test dataset
testDataset = ImageFolder(config.TEST_PATH, testTransform)
# initialize the test data loader
testLoader = get_dataloader(testDataset, config.PRED_BATCH_SIZE)

In [64]:
# initialize the yolov5 using torch hub
yoloModel = torch.hub.load("ultralytics/yolov5", "yolov5s")
# initialize iterable variable
sweeper = iter(testLoader)
# initialize image 
imageInput = []
# grab a batch of test data
print("[INFO] getting the test data...")
batch = next(sweeper)
(images, _) = (batch[0], batch[1])
# send the images to the device
images = images.to(config.DEVICE)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2022-1-5 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)




requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


[INFO] getting the test data...


In [76]:
# loop over all the batch 
for index in range(0, config.PRED_BATCH_SIZE):
	# grab each image
	# rearrange dimensions to channel last and
	# append them to image list
	image = images[index]
	image = image.permute((1, 2, 0))
	imageInput.append(image.cpu().detach().numpy()*255.0)
# pass the image list through the model
print("[INFO] getting detections from the test data...")
results = yoloModel(imageInput, size=300)

[INFO] getting detections from the test data...


In [87]:
# get random index value
randomIndex = random.randint(0,len(imageInput)-1)
# grab index result from results variable
imageIndex= results.pandas().xyxy[randomIndex]
# convert the bounding box values to integer
startX = int(imageIndex["xmin"][0])
startY = int(imageIndex["ymin"][0])
endX = int(imageIndex["xmax"][0])
endY = int(imageIndex["ymax"][0])
# draw the predicted bounding box and class label on the image
y = startY - 10 if startY - 10 > 10 else startY + 10
cv2.putText(imageInput[randomIndex], imageIndex["name"][0],
	(startX, y+10), cv2.FONT_HERSHEY_SIMPLEX,0.65, (0, 255, 0), 2)
cv2.rectangle(imageInput[randomIndex],
	(startX, startY), (endX, endY),(0, 255, 0), 2)
# check to see if the output directory already exists, if not
# make the output directory
if not os.path.exists(config.YOLO_OUTPUT):
    os.makedirs(config.YOLO_OUTPUT)
# show the output image and save it to path
plt.imshow(imageInput[randomIndex]/255.0)
# save plots to output directory
print("[INFO] saving the inference...")
outputFileName = os.path.join(config.YOLO_OUTPUT, "output.png")
plt.savefig(outputFileName)
plt.clf()

[INFO] saving the inference...


In [71]:
from tools.data_utils import normalize
import matplotlib.patches as patches

imageInput = []

In [72]:
# grab a batch of test data
print("[INFO] getting the test data...")

# switch off autograd
with torch.no_grad():
	# loop over all the batch 
	for index in range(0, config.PRED_BATCH_SIZE):
		# grab the image, de-normalize it, scale the raw pixel
		# intensities to the range [0, 255], and change the channel
		# ordering from channels first tp channels last
		image = images[index]
		image = image.permute((1, 2, 0))
		imageInput.append(image.cpu().detach().numpy())

[INFO] getting the test data...


In [73]:
# call the required entry points
ssdModel = torch.hub.load("NVIDIA/DeepLearningExamples:torchhub",
	"nvidia_ssd")
utils = torch.hub.load("NVIDIA/DeepLearningExamples:torchhub", 
	"nvidia_ssd_processing_utils")
# flash model to the device and set it to eval mode
ssdModel.to(config.DEVICE)
ssdModel.eval()
# new list for processed input
processedInput = []
# loop over images and preprocess them
for image in imageInput:
	image = normalize (image)
	processedInput.append(image)
# convert the preprocessed images into tensors
inputTensor = utils.prepare_tensor(processedInput)

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [74]:
# turn off auto-grad
print("[INFO] getting detections from the test data...")
with torch.no_grad():
	# feed images to model
	detections = ssdModel(inputTensor)
# decode the results and filter them using the threshold
resultsPerInput = utils.decode_results(detections)
bestResults = [utils.pick_best(results,
	config.THRESHOLD) for results in resultsPerInput]

[INFO] getting detections from the test data...


In [75]:
# get coco labels 
classesToLabels = utils.get_coco_object_dictionary()
# loop over the image batch
for image_idx in range(len(bestResults)):
	(fig, ax) = plt.subplots(1)
	# denormalize the image and plot the image
	image = processedInput[image_idx] / 2 + 0.5
	ax.imshow(image)
	# grab bbox, class, and confidence values
	(bboxes, classes, confidences) = bestResults[image_idx]

	# loop over the detected bounding boxes
	for idx in range(len(bboxes)):
		# scale values up according to image size
		(left, bot, right, top) = bboxes[idx ] * 300
		# draw the bounding box on the image
		(x, y, w, h) = [val for val in [left, bot, right - left,
			top - bot]]
		rect = patches.Rectangle((x, y), w, h, linewidth=1,
			edgecolor="r", facecolor="none")
		ax.add_patch(rect)
		ax.text(x, y,
			"{} {:.0f}%".format(classesToLabels[classes[idx] - 1],
			confidences[idx] * 100),
			bbox=dict(facecolor="white", alpha=0.5))
	
	# check to see if the output directory already exists, if not
	# make the output directory
	if not os.path.exists(config.SSD_OUTPUT):
		os.makedirs(config.SSD_OUTPUT)
	# save plots to output directory
	print("[INFO] saving the inference {}...".format(image_idx))
	outputFileName = os.path.join(config.SSD_OUTPUT, "output{}.png".format(image_idx))
	plt.savefig(outputFileName)
	plt.clf()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


[INFO] saving the inference 0...
[INFO] saving the inference 1...
[INFO] saving the inference 2...
[INFO] saving the inference 3...


In [52]:
!ls -a .

.   category_names.txt	dataset  sample_data  yolov5s.pt
..  .config		output	 tools


In [88]:
!sudo add-apt-repository -y ppa:git-core/ppa
!sudo apt update
!sudo apt install git -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [

In [89]:
!git config --global user.email "my_email"
!git config --global user.name "my_user"

In [90]:
!git init -b main

Initialized empty Git repository in /content/.git/


In [91]:
!git remote add origin https://<USERNAME>:<Token>@github.com/<USERNAME>/reponame.git

In [ ]:
!git remote -v

In [93]:
!git pull origin main

remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), 1.56 KiB | 801.00 KiB/s, done.
From https://github.com/sc-AhmedAttia/ObjectDetection
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main


In [94]:
!git add output/ tools/ category_names.txt

In [95]:
!git commit -m "Implementation"

[main a5fd253] Implementation
 12 files changed, 110 insertions(+)
 create mode 100644 category_names.txt
 create mode 100644 output/ssd/output0.png
 create mode 100644 output/ssd/output1.png
 create mode 100644 output/ssd/output2.png
 create mode 100644 output/ssd/output3.png
 create mode 100644 output/yolo/output.png
 create mode 100644 output/yolo/output_semi0.png
 create mode 100644 output/yolo/output_semi1.png
 create mode 100644 output/yolo/output_wrong.png
 create mode 100644 tools/__init__.py
 create mode 100644 tools/config.py
 create mode 100644 tools/data_utils.py


In [96]:
!git push origin main

Enumerating objects: 19, done.
Counting objects: 100% (19/19), done.
Delta compression using up to 2 threads
Compressing objects: 100% (17/17), done.
Writing objects: 100% (18/18), 741.52 KiB | 15.13 MiB/s, done.
Total 18 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/sc-AhmedAttia/ObjectDetection.git
   fad144c..a5fd253  main -> main
